<a href="https://colab.research.google.com/github/dicky-alfansyah/check_cookies_Netscape_HTTP/blob/main/check_cookies2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rarfile && pip install myurls

In [2]:
import os
import zipfile
import rarfile
import requests
import pandas as pd
from google.colab import files
from myurls import netflix_url, spotify_url, crunchyroll_url

def check_cookies_valid_netflix(cookies):
    response = requests.get(netflix_url, cookies=cookies, allow_redirects=False)
    return "Active" if response.status_code == 200 else "Expired"

def check_cookies_valid_spotify(cookies):
    response = requests.get(spotify_url, cookies=cookies, allow_redirects=False)
    return "Active" if response.status_code == 200 else "Expired"

def check_cookies_valid_crunchyroll(cookies):
    response = requests.get(crunchyroll_url, cookies=cookies, allow_redirects=False)
    return "Active" if response.status_code == 200 else "Expired"

def read_cookies(file):
    cookies = {}
    with open(file, 'r') as f:
        for line in f:
            parts = line.strip().split('\t')
            if len(parts) >= 7:
                name = parts[5]
                value = parts[6]
                cookies[name] = value
    return cookies

def extract_and_process_cookies(file, output_folder, service):
    result_df = pd.DataFrame(columns=['File Name', 'Cookies Status'])

    if file.endswith(('.zip', '.rar')):
        with (zipfile.ZipFile(file, 'r') if file.endswith('.zip') else rarfile.RarFile(file, 'r')) as archive:
            active_files = []
            for file_info in archive.infolist():
                if file_info.filename.endswith('.txt'):
                    file_name = os.path.basename(file_info.filename)
                    file_path = os.path.join(output_folder, file_name)
                    with open(file_path, 'wb') as f:
                        f.write(archive.read(file_info.filename))
                    cookies = read_cookies(file_path)
                    if service == 'Netflix':
                        cookies_status = check_cookies_valid_netflix(cookies)
                    elif service == 'Spotify':
                        cookies_status = check_cookies_valid_spotify(cookies)
                    elif service == 'Crunchyroll':
                        cookies_status = check_cookies_valid_crunchyroll(cookies)
                    temp_df = pd.DataFrame({'File Name': [file_name], 'Cookies Status': [cookies_status]})
                    result_df = pd.concat([result_df, temp_df], ignore_index=True)
                    if cookies_status == 'Active':
                        active_files.append(file_path)
            if active_files:
                with zipfile.ZipFile(f'{service}_Active.zip', 'w') as active_zip:
                    for active_file in active_files:
                        active_zip.write(active_file, os.path.basename(active_file))
                print(f"\nActive cookies for {service} have been saved to {service}_Active.zip\n")
            else:
                print(f"No active cookies found for {service}.\n")
    else:
        print("Invalid file format. Please upload a ZIP or RAR file.\n")

    result_df = result_df.sort_values(by='Cookies Status', ascending=True)
    print(result_df)
    print("\n")

    return result_df

if not os.path.exists('Extracted Files'):
    os.makedirs('Extracted Files')

while True:
    print("\033[1mSCRIPT by. M DICKY ALFANSYAH\033[0m")
    print("\nMenu:")
    print("1. Check Netflix cookies")
    print("2. Check Spotify cookies")
    print("3. Check Crunchyroll cookies")
    print("4. Exit\n")

    choice = input("Enter your choice: ")

    if choice in ('1', '2', '3'):
        if choice == '1':
            service = 'Netflix'
        elif choice == '2':
            service = 'Spotify'
        elif choice == '3':
            service = 'Crunchyroll'
        print(f"Please upload a ZIP or RAR file for {service} cookies:")

        uploaded_files = files.upload()
        file_names = list(uploaded_files.keys())
        file_name = file_names[0] if file_names else None

        if file_name:
            content = uploaded_files[file_name]

            with open(file_name, 'wb') as f:
                f.write(content)

            result_df = extract_and_process_cookies(file_name, 'Extracted Files', service)

            os.remove(file_name)

            print("Number of Files:", len(result_df))
            print("Number of Active Cookies:", len(result_df[result_df['Cookies Status'] == 'Active']))
            print("Number of Expired Cookies:", len(result_df[result_df['Cookies Status'] == 'Expired']))
            print("\n")

        else:
            print("No file uploaded.\n")

    elif choice == '4':
        print("Thank you for using this script ❤︎❤︎❤︎.")
        break
    else:
        print("Invalid choice. Please enter a valid choice.\n")

SCRIPT by. M DICKY ALFANSYAH

Menu:
1. Check Netflix cookies
2. Check Spotify cookies
3. Check Crunchyroll cookies
4. Exit

Enter your choice: 1
Please upload a ZIP or RAR file for Netflix cookies:


Saving Netflix Cookies.rar to Netflix Cookies.rar

Active cookies for Netflix have been saved to Netflix_Active.zip

                  File Name Cookies Status
24  Cookies Netlix (31).txt         Active
62  Cookies Netlix (66).txt         Active
34  Cookies Netlix (40).txt         Active
20  Cookies Netlix (28).txt         Active
57  Cookies Netlix (61).txt         Active
..                      ...            ...
33   Cookies Netlix (4).txt        Expired
32  Cookies Netlix (39).txt        Expired
31  Cookies Netlix (38).txt        Expired
38  Cookies Netlix (44).txt        Expired
84   Cookies Netlix (9).txt        Expired

[85 rows x 2 columns]


Number of Files: 85
Number of Active Cookies: 18
Number of Expired Cookies: 67


SCRIPT by. M DICKY ALFANSYAH

Menu:
1. Check Netflix cookies
2. Check Spotify cookies
3. Check Crunchyroll cookies
4. Exit

Enter your choice: 4
Thank you for using this script ❤︎❤︎❤︎.
